## Read in config file and do basic checks:

In [ ]:
report_problem()
{
    date | tee -a /tmp/debuglog
    echo "[[FAILED]] $*" | tee -a /tmp/debuglog
    exit 1 # exit the bash kernel (which does not exit nbconvert :-( ))
}

source ~/jh_test.config || report_problem "problem reading config file"

check_ssh()
{
    [ "$(echo $SSHCMD)" != "" ] || report_problem "SSHCMD is not set"
    [ "$($SSHCMD echo OK)" = "OK" ] || report_problem "SSHCMD ($SSHCMD) did not connect successfully"
}

check_targetdir()
{
    $SSHCMD "[ -d '$TARGETDIR' ]" || report_problem "The directory ($TARGETDIR) does not exist on the server"
}

check_bridge()
{
    [ "$(echo $BRIDGENAME)" != "" ] || report_problem "BRIDGENAME is not set"
    [[ "$($SSHCMD /sbin/brctl show "$BRIDGENAME" 2>&1 1>/dev/null)" == "" ]] || report_problem "bridge ($BRIDGENAME) not found"
}

check_ssh
check_targetdir
check_bridge
echo "Config checks OK"

## Run the next cell to test if build has been completed:
It should take about 20 seconds to run.

In [ ]:
echo "Checking if all build steps have been done..."
$SSHCMD <<<"set -- $TARGETDIR/installdir/toplevel-kvm-tap-build.sh check ; $(cat .utilities/bashsteps-check-next-step-to-do.sh)"
[ "$?" = 0 ] && echo "[[FAILED]] Some build steps still need to be done." || echo "All steps have been done.  The build has been completed."

## Run the next cell to test if all private network addresses work:

In [ ]:
allip="$($SSHCMD <<<'for ddc in "'$TARGETDIR'/installdir/jhvmdir"*/datadir.conf; do ( source $ddc; echo "$VMIP") ; done')"
allok=true
for ip in $allip ; do
    $SSHCMD "$TARGETDIR/installdir/jhvmdir/ssh-shortcut.sh" -q ping -qc 1 $ip || {
       echo "[[FAILED]] No connection to $ip"
       allok=false
    }
    echo
done
$allok && echo "All private network addresses work."

## Run the next cell to test the connection to the login screen:

In [ ]:
hubport="$($SSHCMD <<<'source "'$TARGETDIR'/installdir/jhvmdir-hub/datadir.conf"; echo "$VNCPORT"')"
hubip="${SSHCMD#*@}"
hubip="${hubip%% *}"
echo "Testing connection at $hubip:$(( hubport + 90 ))"
curl --max-time 5 -k https://$hubip:$(( hubport + 90 ))/php/login.php 2>/dev/null | grep -q 'Sign In'
echo -n "Connection to login page "
[ "$?" = 0 ] && echo "succeeded." || echo "[[FAILED]]."